In [ ]:
#Regression model modeling that predicts the progress of the patient's disease after one year based on the patient's data
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.datasets import load_diabetes

diabetes_data = load_diabetes()

#numpy로 먼저 가져온 다음에 tensor로 변환
x = torch.from_numpy(np.array(diabetes_data.data[:,:-1],dtype = np.float32))#target열 포함 총 11개의 col이 있는데 target 열(맨 마지막)을 빼고 가져오겠다
y = torch.from_numpy(np.array(diabetes_data.data[:,[-1]],dtype = np.float32))#target 열(맨 마지막)만 가져오겠다,[-1] 사용이유는 1차원(벡터)가 아닌 2차원 형태로 가져 오기 위함
print(x.shape)
print(y.shape)
print(y)


In [ ]:
# regression-model modeling
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self,input_dim,out_dim):
    super().__init__()
    self.linear = nn.Linear(input_dim,out_dim)

  def forward(self,x):
    return self.linear(x)
print(x.size(1),y.size(1))
model = Net(x.size(1),y.size(1)) #각 9,1이 들어감x,y의 1번 째 인덱스가 들어가는 것


In [ ]:
# training
lr = 0.01
epoch_num = 10000
optimizer = torch.optim.SGD(model.parameters(),lr = lr)

for epoch in range(epoch_num+1):
  out = model(x)
  if epoch == 0:
    #out.shape는 y.shape가 동일
    print(out.shape)
  loss = F.mse_loss(out,y)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
# check containing parameters
print(loss)
for param in model.parameters():
  print(param)

In [ ]:
# install plotly
!pip install plotly

In [ ]:
# visualize the distribution of the predicted values and the actual values
df = pd.DataFrame(torch.cat([y,out],dim=1).detach().numpy(),columns=['y','out'])

import plotly.graph_objects as go
import plotly.offline as pyo # jupyter notebook 에서 보여지도록 설정하는 부분 (가끔 안나올 때, 이 명령을 하면 됨)
pyo.init_notebook_mode()

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['y'], mode='markers', name='y'))
fig.add_trace(go.Scatter(x=df.index, y=df['out'], mode='markers', name='out'))

fig.update_layout(yaxis_range=[-0.5,0.5])
fig.show()